# Pie charts with number of hotspots per signature in Fig 2B

In [1]:
from collections import defaultdict
import gzip

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

In [3]:
data_dir = './data'

In [4]:
sigs_color_dict = {
'SBS1': '#1e79baff',
 'SBS5': '#7ebdeaff',
 'SBS17a': '#ffbb78ff',
 'SBS17b': '#ff8921ff',
 'SBS18': '#9467bdff',
 'SBS40': '#afd6f2ff',
 'Other': '#d5d5d5ff',
    'SBS7a': '#8c564bff', 
    'SBS7b': '#c49c94ff', 
    'SBS93': '#c5b0d5ff'
}

In [5]:
def pie_chart_signatures(ctype, signature_hotspots, n_hotspots, output, sigs_color, close=False): 
    """Plot a pie chart with the proportion of signatures creating hotspots in a cancer type"""
    
    # Transform to pie chart
    labels = []
    ring1 = []
    ring1_colors = []
    total_count = sum(signature_hotspots.values())
    for sig, count in sorted_sigs:
        fraction = count / total_count
        labels.append(f'{sig} ({100*fraction:.2}%)')
        ring1.append(fraction)
        ring1_colors.append(sigs_color.get(sig, sigs_color['Other']))
        
    # Plot
    fig = plt.figure(figsize=(1.25, 1.25))
    ax = plt.subplot2grid((1, 1), (0, 0))
    size = 0.5
    plt.title(ctype, fontsize=7)


    ax.pie(
        ring1, 
        radius=1.25, 
        colors=ring1_colors, 
        startangle=90,
        wedgeprops=dict(width=size), 
        textprops={'fontsize': 12}
    )
    
    ax.text(
        0.5, 0.5, 
        f'{n_hotspots:,}\nhotspots', 
        horizontalalignment='center', 
        verticalalignment='center', 
        transform=ax.transAxes,
        fontsize=6
    )
    
    if output: 
        plt.savefig(output, bbox_inches='tight', dpi=300)
    
    if close: 
        plt.close()

## Panel 2B

In [6]:
ctypes_to_plot = [
    'COADREAD',
    'ESOPHA_STOMACH',
    'SKCM'
]

In [ ]:
for ctype in ctypes_to_plot: 
    
    # Load signature probabilities per hotspot
    input_f = f'{data_dir}/hotspots_SBS_prob__{ctype}.txt.gz'
       
    # Read signature probabilities per mutation in a hotspot
    hotspot_sigs = defaultdict(lambda: defaultdict(float))
    signatures = set()
    with gzip.open(input_f, 'rt') as fd: 
        next(fd)
        for line in fd: 
            hotspot, _, _, _, _, signature, prob = line.strip().split()
            hotspot_sigs[hotspot][signature] = prob
            signatures.add(signature)
    
    # Assign hotspot to signature by maximum likelihood
    signature_hotspots = defaultdict(int)
    for hotspot, data in hotspot_sigs.items(): 
        sorted_sigs = sorted([(s, float(p)) for s, p in data.items()], key=lambda x: x[1], reverse=True)
        if sorted_sigs[0][1] > 0: 
            signature_hotspots[sorted_sigs[0][0]] += 1
    
    # Sort signatures by number of hotspots
    sorted_sigs = sorted([(s, c) for s, c in signature_hotspots.items()], key=lambda x: x[1], reverse=True)
    
    # Plot
    for extension in ['png']: 
        output = f'./fig_2B_{ctype}.{extension}'
        pie_chart_signatures(ctype, signature_hotspots, len(hotspot_sigs.keys()), output, sigs_color_dict)
    
    print(ctype)

COADREAD
ESOPHA_STOMACH
